# In this notebook, we would be using ChatGPT in order to understand the user's requirements and extract all the relevant attributes of the clothing from his query

### Importing libraries

In [1]:
import os
from dotenv import load_dotenv 
import openai

In [2]:
os.getcwd()

'D:\\Projects\\Flipkart\\Fashion_Recommender_System'

### Loading OpenAI api-key

In [3]:
load_dotenv()
openai.api_key = os.getenv('api_key')

### Accepting the query of the user

In [4]:
user_query = input("Enter your search query: ")

Enter your search query: I am looking for a shirt of green colour and size XL for men that can be worn for parties. It should be plain and not check. It should be long sleeved with rating of at-least 4. The price should be around Rs 700. It should be long sleeved and of brand Adidas. 


### Handling the request limit for ChatGPT (Max 3 requests per 1 minute interval)

In [5]:
def Limit_Max_Requests_per_minute():
    global total_requests_till_now
    global last_request_time
    
    if(total_requests_till_now == 0 or time.time() - last_request_time > 60 + 1):      # 60s = 1 min (+1 for safe side)
        last_request_time = time.time()
        total_requests_till_now = 1

    elif(time.time() - last_request_time <= 60 and total_requests_till_now < 3):   # max no. of requests = 3 per min 
        total_requests_till_now += 1

    else:                                                                  # wait for 1 min to be over for next API request.
        time_to_wait = (60+1) - (time.time() - last_request_time)      
        # +1s was done to compromise in some cases where python rounds off 45.2743492493243289 to 45.
        
        print(colored("Please wait for {} seconds.....".format(time_to_wait), 'red'))
        time.sleep(time_to_wait)
        last_request_time = time.time()
        total_requests_till_now = 1

### Extracting the clothing type desired by the user 

In [6]:
def Extract_Category(user_query): 
    
    instruction = """You are an intelligent information extraction bot. You are provided with a context. The context talks about one of the following clothing outfit:
    1) hat
    2) outwear
    3) pants
    4) shirt
    5) shoes
    6) shorts
    7) skirt
    8) t-shirt
    
    You have to analyze the context and predict which of the above 8 categories is being talked about in the context. 
    
    Return format: Only return a single word string !!! 
    """
    
    message = """
    
    %Context
    {user_query}
    
    %Your Answer
    """
    
    
    message = message.format(user_query=user_query)

    prompt = [
        {"role": "system", "content": instruction},
        {"role": "user", "content": message},
    ]


    Limit_Max_Requests_per_minute()

    response = openai.ChatCompletion.create(
        model='gpt-3.5-turbo',
        messages=prompt,
        temperature=0.7
    )   

    response_message = response["choices"][0]["message"]["content"]
    
    return response_message

In [7]:
category = Extract_Category(user_query)
category

'Shirt'

### Extracting the clothing attributes mentioned by the user

In [8]:
def Extract_Attributes(user_query): 
    
    instruction = """You are an intelligent information extraction bot. You are provided with a context. The context mentions the following 12 listed attributes (as keys) along with the possible values they can take (as set) of a Shirt clothing: 
    1) Brand: {'Adidas', 'Nike', 'Puma', 'Lyra', 'Peter England', 'Zara', 'Calvin Klein', 'Express', 'Arrow', 'Raymond', 'Levi's'}
    2) Size: {'S', 'M', 'L', 'XL', XXL'}
    3) Color: {'Red', 'Blue', 'Green', 'Pink', 'Yellow', 'Purple', 'Navy', 'Black', 'Maroon', 'White', 'Grey'}
    4) Regional Existence: {'North India', 'South India', 'North East India', 'West India', 'Punjab', 'Haryana', 'Kerala', 'Tamil Nadu', 'Gujarat and Rajasthan', 'Kashmir, Himachal and Uttarakhand', 'Kerala and Tamil Nadu', 'All over India'}
    5) Gender: {'Male', 'Female', 'Unisex'}
    6) Body Type: {'Slim', 'Athletic/Muscular', 'Regular' 'Tall', 'Short'}
    7) Occasion: {'Casual', 'Business-Casual', 'Formal', 'Special Events', 'Cultural and Festive Celebrations', 'Travel', 'Outdoor Activities'}
    8) Price: An integer value
    9) Age: {'less than 15 years', '15-20 years' '20-30 years', 'above 30 years', 'above 50 years'}
    10) Style: {'Dress Shirt', 'Button-Down Shirt', 'Polo Shirt', 'Check Shirt', 'Plain Shirt'}
    11) Sleeves: {'Long sleeved', 'Sleeveless'}
    12) Rating: A floating value lying between 1 to 5
    
    You have to analyze the context and extract the above mentioned attributes. If the exact value of an attribute is not present in the context, select the one from the select most closer to the one mentioned in the context. And if any attribute is not at all mentioned in the context (either directly or indirectly), write 'NULL' to its key.
    
    Return format: Only return a dictionary with keys as all the above mentioned attributes and values as the information extracted from the context or NULL (if not listed in the context). !!! 
    """
    
    message = """
    
    %Context
    {user_query}
    
    %Your Answer
    """
    
    
    message = message.format(user_query=user_query)

    prompt = [
        {"role": "system", "content": instruction},
        {"role": "user", "content": message},
    ]


    Limit_Max_Requests_per_minute()

    response = openai.ChatCompletion.create(
        model='gpt-3.5-turbo',
        messages=prompt,
        temperature=0.7
    )   

    response_message = response["choices"][0]["message"]["content"]
    
    return response_message

In [10]:
attributes = Extract_Attributes(user_query)
attributes

{'Brand': 'Adidas',
 'Size': 'XL',
 'Color': 'Green',
 'Regional Existence': 'NULL',
 'Gender': 'Male',
 'Body Type': 'NULL',
 'Occasion': 'Special Events',
 'Price': 700,
 'Age': 'NULL',
 'Style': 'Plain Shirt',
 'Sleeves': 'Long sleeved',
 'Rating': 4.0}

### Now since we have extracted the attributes, we can perform similarity search using KNN to fetch top most similar clothing outfits as per the requirements of the user